In [1]:
!pip install kafka-python
!pip install redis

In [2]:
import os
import requests
import json
import redis
import pyspark
import urllib
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from datetime import datetime

In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[2] pyspark-shell'
conf = SparkConf().set("spark.jars","/home/jovyan/work/spark-streaming-kafka-0-8-assembly_2.11-2.4.4.jar")
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc,60) # 60 is the batch interval :)
debug = True
saveToLocalDisk = True
spark = SparkSession(sc)

In [4]:
def SetVal(x):
    r = redis.StrictRedis(host = 'redis-store', port = 6379)
    r.set('LatestNews',"{\"tasks\":["+str(x)+"]}")
        
def SaveToNewsFile(rdd):
    if not rdd.isEmpty():
        rdd.toDF( [ "Timestamp", "Title","Summary","Link" ] ) \
        .write.save("/home/jovyan/work/NewsData.csv",
                    maxRecordsPerFile=1000, format="csv", mode="append") 
        

In [ ]:
kafkaStream = KafkaUtils.createStream(
    ssc=ssc, 
    zkQuorum='zookeeper:2181', 
    groupId='test-consumer-group',#, #Consumer Group 
    topics={'news_feed':1})

olines = kafkaStream.map(lambda x: x[1]).map(lambda a: json.loads(a))
    
olines = olines.filter(lambda a: ('Type' in a) & (a['Type']=="News"))
    

if(saveToLocalDisk):
     olines.map(lambda a: (a['Timestamp'],a['Title'],a['Summary'],a['Link'])) \
        .foreachRDD(SaveToNewsFile)

olines = olines.map(lambda a: json.dumps(a)).reduce(lambda v,agg: agg+", "+v)

olines = olines.map(lambda x: SetVal(x))
if(debug):
    olines.pprint()
    
ssc.start()  
print("Going to wait termination")
ssc.awaitTermination()

Going to wait termination
-------------------------------------------
Time: 2019-12-07 23:43:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-07 23:44:00
-------------------------------------------
None

-------------------------------------------
Time: 2019-12-07 23:45:00
-------------------------------------------
None

-------------------------------------------
Time: 2019-12-07 23:46:00
-------------------------------------------
None

-------------------------------------------
Time: 2019-12-07 23:47:00
-------------------------------------------
None

-------------------------------------------
Time: 2019-12-07 23:48:00
-------------------------------------------
None

-------------------------------------------
Time: 2019-12-07 23:49:00
-------------------------------------------
None

-------------------------------------------
Time: 2019-12-07 23:50:00
-------------------------------------------
None

-------------------

In [ ]:
lines = lines.map(lambda a: requests.post(
        "http://serving:8501/v1/models/half_plus_two:predict",
        "{\"instances\": ["+a+"]}").text)
if(debug):
    lines.pprint()

lines = lines.flatMap(lambda x: json.loads(x)["predictions"])
if(debug):
    lines.pprint()

lines = lines.map(lambda x: SetVal(x))
if(debug):
    lines.pprint()